<a href="https://colab.research.google.com/github/ra1ph2/Vision-Transformer/blob/main/VisionTransformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Libraries Imported

In [2]:
import torch
from torch import nn
from torch import functional as F

#### Model Architecture

In [38]:
class Attention(nn.Module):
    def __init__(self, embed_dim, heads=8, activation=None, dropout=0.1):
        super(Attention, self).__init__()
        self.heads = heads
        self.embed_dim = embed_dim
        self.query = nn.Linear(embed_dim, embed_dim)
        self.key = nn.Linear(embed_dim, embed_dim)
        self.value = nn.Linear(embed_dim, embed_dim)
        self.softmax = nn.Softmax(dim=-1)
        if activation == 'relu':
            self.activation = nn.ReLU()
        else:
            self.activation = nn.Identity()
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, inp):
        # inp: (batch_size, seq_len, embed_dim)
        batch_size, seq_len, embed_dim = inp.size()
        assert embed_dim == self.embed_dim

        query = self.activation(self.query(inp))
        key   = self.activation(self.key(inp))
        value = self.activation(self.value(inp))

        # output of _reshape_heads(): (batch_size * heads, seq_len, reduced_dim) | reduced_dim = embed_dim // heads
        query = self._reshape_heads(query)
        key   = self._reshape_heads(key)
        value = self._reshape_heads(value)

        # attention_scores: (batch_size * heads, seq_len, seq_len) | Softmaxed along the last dimension
        attention_scores = self.softmax(torch.matmul(query, key.transpose(1, 2)))

        # out: (batch_size * heads, seq_len, reduced_dim)
        out = torch.matmul(self.dropout(attention_scores), value)
        
        # output of _reshape_heads_back(): (batch_size, seq_len, embed_size)
        out = self._reshape_heads_back(out)

        return out, attention_scores

    def _reshape_heads(self, inp):
        # inp: (batch_size, seq_len, embed_dim)
        batch_size, seq_len, embed_dim = inp.size()

        reduced_dim = self.embed_dim // self.heads
        assert reduced_dim * self.heads == self.embed_dim
        out = inp.reshape(batch_size, seq_len, self.heads, reduced_dim)
        out = out.permute(0, 2, 1, 3)
        out = out.reshape(-1, seq_len, reduced_dim)

        # out: (batch_size * heads, seq_len, reduced_dim)
        return out

    def _reshape_heads_back(self, inp):
        # inp: (batch_size * heads, seq_len, reduced_dim) | reduced_dim = embed_dim // heads
        batch_size_mul_heads, seq_len, reduced_dim = inp.size()
        batch_size = batch_size_mul_heads // self.heads

        out = inp.reshape(batch_size, self.heads, seq_len, reduced_dim)
        out = out.permute(0, 2, 1, 3)
        out = out.reshape(batch_size, seq_len, self.embed_dim)

        # out: (batch_size, seq_len, embed_dim)
        return out

In [40]:
attention = Attention(embed_dim=4, heads=2, activation=None)
attention_lib = nn.MultiheadAttention(embed_dim=4, num_heads=2)
inp = torch.ones((1, 2, 4))
print(inp)
out, wts = attention(inp)
print(out)
print(wts)
out_lib, out_wts = attention_lib(inp.permute(1, 0, 2), inp.permute(1, 0, 2), inp.permute(1, 0, 2))
print(out_lib.permute(1, 0, 2))
print(out_wts)

tensor([[[1., 1., 1., 1.],
         [1., 1., 1., 1.]]])
tensor([[[-0.1458, -0.9517, -0.5752,  0.0712],
         [-0.0729, -0.4758, -0.2876,  0.0356]]], grad_fn=<UnsafeViewBackward>)
tensor([[[0.5000, 0.5000],
         [0.5000, 0.5000]],

        [[0.5000, 0.5000],
         [0.5000, 0.5000]]], grad_fn=<SoftmaxBackward>)
tensor([[[-0.4409,  0.7654,  0.4589, -0.5832],
         [-0.4409,  0.7654,  0.4589, -0.5832]]], grad_fn=<PermuteBackward>)
tensor([[[0.5000, 0.5000],
         [0.5000, 0.5000]]], grad_fn=<DivBackward0>)


In [32]:
# Check if Dropout should be used after second Linear Layer
class FeedForward(nn.Module):
    def __init__(self, embed_dim, forward_expansion=1, dropout=0.1):
        super(FeedForward, self).__init__()
        self.embed_dim = embed_dim
        self.fc1 = nn.Linear(embed_dim, embed_dim * forward_expansion)
        self.activation = nn.GELU()
        self.fc2 = nn.Linear(embed_dim * forward_expansion, embed_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, inp):
        # inp: (batch_size, seq_len, embed_dim)
        batch_size, seq_len, embed_dim = inp.size()
        assert embed_dim == self.embed_dim

        out = self.dropout(self.activation(self.fc1(inp)))
        # out = self.dropout(self.fc2(out))
        out = self.fc2(out)

        # out: (batch_size, seq_len, embed_dim)
        return out 

In [33]:
ff = FeedForward(8, 1)
inp = torch.ones((1, 2, 8))
print(inp)
out = ff(inp)
print(out)

tensor([[[1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1.]]])
tensor([[[-0.0409, -0.0025, -0.0943, -0.0105, -0.0891, -0.0134, -0.1431,
           0.3287],
         [-0.0265, -0.0158, -0.0895,  0.0139, -0.0681, -0.0181, -0.1407,
           0.3108]]], grad_fn=<AddBackward0>)


In [9]:
class TransformerBlock(nn.Module):
    def __init__(self, embed_dim, heads=8, activation=None, forward_expansion=1, dropout=0.1):
        super(TransformerBlock, self).__init__()
        self.embed_dim = embed_dim
        self.norm1 = nn.LayerNorm(embed_dim)
        self.attention = Attention(embed_dim, heads, activation, dropout)
        self.norm2 = nn.LayerNorm(embed_dim)
        self.feed_forward = FeedForward(embed_dim, forward_expansion, dropout)

    def forward(self, inp):
        # inp: (batch_size, seq_len, embed_dim)
        batch_size, seq_len, embed_dim = inp.size()
        assert embed_dim == self.embed_dim

        res = inp
        out = self.norm1(inp)
        out, _ = self.attention(out)
        out = out + res
        
        res = out
        out = self.norm2(out)
        out = self.feed_forward(out)
        out = out + res

        # out: (batch_size, seq_len, embed_dim)
        return out

In [34]:
tb = TransformerBlock(8, 2)
inp = torch.ones((1, 2, 8))
print(inp)
out = tb(inp)
print(out)

tensor([[[1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1.]]])
tensor([[[1.2714, 1.6808, 1.0581, 1.8078, 0.4354, 1.0091, 0.9663, 1.1954],
         [1.2904, 1.6790, 1.0655, 1.7437, 0.5846, 1.0650, 0.9050, 1.2332]]],
       grad_fn=<AddBackward0>)


In [36]:
class Transformer(nn.Module):
    def __init__(self, embed_dim, layers, heads=8, activation=None, forward_expansion=1, dropout=0.1):
        super(Transformer, self).__init__()
        self.embed_dim = embed_dim
        self.trans_blocks = nn.ModuleList(
            [TransformerBlock(embed_dim, heads, activation, forward_expansion, dropout) for i in range(layers)]
        )

    def forward(self, inp):
        # inp: (batch_size, seq_len, embed_dim)

        out = inp
        for block in self.trans_blocks:
            out = block(out)

        # out: (batch_size, seq_len, embed_dim)
        return out

In [37]:
tb = TransformerBlock(8, 2)
trans = Transformer(8, 1, 2)
inp = torch.ones((1, 2, 8))
print(inp)
out = tb(inp)
print(out)
out = trans(inp)
print(out)

tensor([[[1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1.]]])
tensor([[[0.9370, 0.9729, 1.1449, 1.0035, 0.4312, 0.4503, 1.0191, 0.7539],
         [0.8959, 1.1752, 1.3447, 1.2614, 0.3136, 0.2084, 0.8908, 0.8482]]],
       grad_fn=<AddBackward0>)
tensor([[[0.8253, 1.3689, 1.1571, 0.5761, 0.8079, 0.9461, 1.2345, 0.2657],
         [0.8667, 1.3327, 1.1235, 0.6152, 0.8009, 0.9217, 1.2245, 0.2974]]],
       grad_fn=<AddBackward0>)


In [30]:
# Not Exactly Same as Paper | Check if Dropout should be used here
class ClassificationHead(nn.Module):
    def __init__(self, embed_dim, classes, dropout=0.1):
        super(ClassificationHead, self).__init__()
        self.embed_dim = embed_dim
        self.classes = classes
        self.fc1 = nn.Linear(embed_dim, embed_dim // 2)
        self.activation = nn.GELU()
        self.fc2 = nn.Linear(embed_dim // 2, classes)
        self.softmax = nn.Softmax(dim=-1)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, inp):
        # inp: (batch_size, embed_dim)
        batch_size, embed_dim = inp.size()
        assert embed_dim == self.embed_dim

        out = self.dropout(self.activation(self.fc1(inp)))
        out = self.softmax(self.fc2(out))

        # out: (batch_size, embed_dim) | SoftMaxed along the last dimension
        return out

In [31]:
ch = ClassificationHead(8, 2)
inp = torch.ones((2, 8))
print(inp)
out = ch(inp)
print(out)

tensor([[1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])
tensor([[0.6724, 0.3276],
        [0.6441, 0.3559]], grad_fn=<SoftmaxBackward>)


In [53]:
class VisionTransformer(nn.Module):
    def __init__(self, patch_size, max_len, embed_dim, classes, layers, heads=8, activation=None, forward_expansion=1, dropout=0.1):
        super(VisionTransformer, self).__init__()
        self.patch_size = patch_size
        self.embed_dim = embed_dim
        self.patch_to_embed = nn.Linear(patch_size * patch_size * 3, embed_dim)
        self.position_embed = nn.Parameter(torch.randn((max_len, embed_dim)))
        self.transformer = Transformer(embed_dim, layers, heads, activation, forward_expansion, dropout)
        self.classification_head = ClassificationHead(embed_dim, classes)

    def forward(self, inp):
        # inp: (batch_size, seq_len, patch_size, patch_size, 3) | seq_len would be (width*height)/(patch_size**2)
        batch_size, seq_len, patch_size, _, channels = inp.size()
        assert patch_size == self.patch_size
        assert channels == 3

        out = inp.reshape(batch_size, seq_len, -1)
        out = self.patch_to_embed(out)
        # out: (batch_size, seq_len, embed_dim)

        class_token = torch.randn((batch_size, 1, self.embed_dim))
        out = torch.cat([class_token, out], dim=1)
        # out: (batch_size, seq_len+1, embed_dim)

        position_embed = self.position_embed[:seq_len+1]
        position_embed = position_embed.unsqueeze(0).expand(batch_size, seq_len+1, self.embed_dim)
        out = out + position_embed
        # out: (batch_size, seq_len+1, embed_dim) | Added Positional Embeddings

        out = self.transformer(out)
        # out: (batch_size, seq_len+1, embed_dim) 
        class_token = out[:, 0]
        # class_token: (batch_size, embed_dim)

        class_out = self.classification_head(class_token)
        # class_out: (batch_size, classes)
        
        return class_out, out

In [54]:
vit = VisionTransformer(2, 3, 8, 2, 2)
inp = torch.ones((2, 2, 2, 2, 3))
print(inp)
class_out, out = vit(inp)
print(class_out)
print(class_out.shape)
print(out)
print(out.shape)

tensor([[[[[1., 1., 1.],
           [1., 1., 1.]],

          [[1., 1., 1.],
           [1., 1., 1.]]],


         [[[1., 1., 1.],
           [1., 1., 1.]],

          [[1., 1., 1.],
           [1., 1., 1.]]]],



        [[[[1., 1., 1.],
           [1., 1., 1.]],

          [[1., 1., 1.],
           [1., 1., 1.]]],


         [[[1., 1., 1.],
           [1., 1., 1.]],

          [[1., 1., 1.],
           [1., 1., 1.]]]]])
tensor([[0.7722, 0.2278],
        [0.5310, 0.4690]], grad_fn=<SoftmaxBackward>)
torch.Size([2, 2])
tensor([[[-0.1050, -0.3875, -1.4965, -1.2352, -2.4860,  0.8516,  0.5158,
          -2.4766],
         [-0.4389, -0.4218,  1.7999, -0.8428,  0.4959, -0.5753,  0.6173,
          -1.8818],
         [-0.5806,  1.7895,  2.2520, -1.6987, -1.4750,  1.9487,  0.4231,
          -0.1351]],

        [[ 2.2784,  1.2812,  1.4759, -0.6443, -3.0957, -2.5200,  2.0057,
          -0.5107],
         [ 0.1411, -0.2414,  0.9622, -1.1537,  0.7594, -0.6575,  1.3814,
          -1.6858],
        